# 네이버 매장정보 크롤링

### naver_매장존재확인_크롤링에서 만든 파일을 사용합니다

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  
import chromedriver_autoinstaller


# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
from tqdm import tqdm  

데이터 가져오기

In [2]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
res_df['검색어'] = res_df['상세주소'] + " " + res_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
res_df = res_df[column_order]
res_df.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [3]:
이름 = '해민' # 본인 이름을 입력하세요

# 데이터 불러오기
original_res_df = pd.read_excel(f'data/restaurant_info_data(2)/{이름}.xlsx')
original_df = original_res_df.copy()
original_df.head()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명,데이터기준일,담당자
0,"선릉로82길 23, 브롱스",기타,브롱스,브롱스 선릉점,서울특별시,강남구,"선릉로82길 23,",서울특별시 강남구 대치동 896-3,"서울특별시 강남구 선릉로82길 23, 3층 (대치동, 리치빌딩)",204520.809001,444535.969338,가능,영업/정상,2024-02-01,해민
1,"삼성로84길 15, 김동진장호왕곱창",한식,김동진장호왕곱창,김동진장호왕곱창,서울특별시,강남구,"삼성로84길 15,",서울특별시 강남구 대치동 957-7번지,"서울특별시 강남구 삼성로84길 15, 지하1층 (대치동)",205133.129847,444826.116205,가능,영업/정상,2024-02-01,해민
2,삼성로81길 23 춘하추동밀면,한식,춘하추동밀면,춘하추동밀면 대치직영점,서울특별시,강남구,삼성로81길 23,서울특별시 강남구 대치동 901-65,서울특별시 강남구 삼성로81길 23 (대치동),204864.262676,444652.961363,가능,영업/정상,2024-02-01,해민
3,"선릉로131길 19, 앙마꼬치",기타,앙마꼬치,앙마꼬치,서울특별시,강남구,"선릉로131길 19,",서울특별시 강남구 논현동 118-2,"서울특별시 강남구 선릉로131길 19, 지하1층 (논현동)",203407.179297,446115.386972,가능,영업/정상,2024-02-01,해민
4,"도산대로94길 20, 커피엠스테이블",기타,커피엠스테이블,커피엠스테이블(COFFEE M STABLE),서울특별시,강남구,"도산대로94길 20,",서울특별시 강남구 청담동 57-2번지,"서울특별시 강남구 도산대로94길 20, 지상1층 (청담동)",204493.269617,446843.324331,가능,영업/정상,2024-02-01,해민


한번에 몇개씩 크롤링할지 결정

In [4]:
start_num = 2000 # 할 차례, 여기를 계속 수정해주세요. 
num = 100  # 몇개씩 크롤링할지(기본은 100개를 추천합니다. -> 16분)
end_num = start_num + num
res_df = original_df[start_num:] # 맨 마지막 크롤링할떄는 end_num을 지우세요. [start_num:] <-이렇게 
res_df = res_df.reset_index(drop=True)
res_df.head(3)

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명,데이터기준일,담당자
0,일원로9길 6 치어스,일식,치어스,치어스,서울특별시,강남구,일원로9길 6,서울특별시 강남구 일원동 687-1,서울특별시 강남구 일원로9길 6 (일원동),207182.470517,443019.043050,가능,영업/정상,2024-02-01,해민
1,언주로150길 50 청사포산꼼장어,경양식,청사포산꼼장어,청사포산꼼장어,서울특별시,강남구,언주로150길 50,서울특별시 강남구 논현동 95-7번지,서울특별시 강남구 언주로150길 50 (논현동),203312.954546,446638.270970,가능,영업/정상,2024-02-01,해민
2,"압구정로 152, 비엔나커피하우스",기타,비엔나커피하우스,비엔나커피하우스 압구정극동타워점,서울특별시,강남구,"압구정로 152,",서울특별시 강남구 신사동 575번지,"서울특별시 강남구 압구정로 152, 1층 110, 111, 112, 113, 114...",202231.679116,447037.821825,가능,영업/정상,2024-02-01,해민


driver 세팅

In [5]:
# # # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정 
# driver = webdriver.Chrome(ChromeDriverManager().install())


# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

크롤링 시작

In [6]:
# 가져올 정보들에 대한 리스트 지정
# 가게명, 업태명, 주소, 주력메뉴(맨첫번째메뉴) + 가격까지, 방문자리뷰수,블로그리뷰수

restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []
visitor_review_list =[]
blog_review_list =[]

# 크롤링 시작합니다.
total_iterations = len(res_df) # 크롤링 진행 과정을 파악하기 위해 tqdm 사용 
for i in tqdm(range(total_iterations), desc='진행 중', position=0, leave=True):


    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
        
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            
            # 가게 이름, 변수로 지정
            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려

            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
            # 리뷰 긁어오기 
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
            
            visitor_review = 0
            blog_review = 0
            # PXMot 가 존재할 때 (별점 방문자 블로그 중  뭐라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0      
        else: 
            # searchIframe 프레임 
            # searchIframe 프레임으로 가야하는 경우는 검색했을 때 결과가 바로 안나오고 식당리스트가 먼저나오는 경우
            # 그래서 searchIFrame에서 첫번쨰 식당을 선택하고 그다음에 entryIFrame으로 들어가야함
            searchIframe = driver.find_element(By.ID,'searchIframe')
            driver.switch_to.frame(searchIframe)


            ## 클릭 
            driver.find_element(By.CLASS_NAME,'YwYLL').click()
            
            # 기본프레임 
            time.sleep(1)
            driver.switch_to.default_content()
        

            # entryIframe 프레임 
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)

            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text

            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려

            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"

            # 리뷰 수 긁어오기
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
        
            # PXMot 가 존재할 때 (별점 방문자 블로그 중  뭐라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '').strip()
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0        
    ## 예외처리 : vscode 상태가 안좋아서 그냥 검색이 안된경우 이거나 진짜 없는 식당일 때.
    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        visitor_review = 0
        blog_review = 0

        print(f"에러 메시지: {str(e)}")


    ## 리스트에 추가
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    visitor_review_list.append(visitor_review)
    blog_review_list.append(blog_review)

    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('방문자리뷰 : ',visitor_review)
    print('블로그리뷰 : ',blog_review)
    print('-----------------------------')
res_info_df = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list,
                             '방문자리뷰':visitor_review_list,
                             '블로그리뷰':blog_review_list,
                             })
driver.quit()

## 결과 데이터에 검색어 칼럼 추가
## 추후에 인덱스로 쓸 예정이기 떄문

res_info_df['검색어'] = res_df['검색어']
res_info_df.to_excel(f'./data/restaurant_info_data(2)/restaurant_info_data/restaurant_tag_df_{이름}_{start_num}_{end_num}.xlsx', index=False)

# 100개 기준 16분 

진행 중:   0%|          | 0/74 [00:00<?, ?it/s]

진행 중:   1%|▏         | 1/74 [00:07<09:41,  7.97s/it]

식당 이름: 치어스 강남일원점
식당 카테고리: 맥주,호프
주소: 서울 강남구 일원로9길 6 1층 치어스 강남일원점
대표 메뉴: 치킨 플래터
가격: 25,000원
방문자리뷰 :  97
블로그리뷰 :  5
-----------------------------


진행 중:   3%|▎         | 2/74 [00:15<08:57,  7.47s/it]

식당 이름: 청사포산꼼장어
식당 카테고리: 장어,먹장어요리
주소: 서울 강남구 언주로150길 50 진명빌딩
대표 메뉴: 청사포세트 2인
가격: 71,000원
방문자리뷰 :  37
블로그리뷰 :  68
-----------------------------


...

식당 이름: 경성양꼬치 선릉직영점
식당 카테고리: 양꼬치
주소: 서울 강남구 삼성로85길 39 가리온
대표 메뉴: 고급갈비
가격: 32,000원
방문자리뷰 :  169
블로그리뷰 :  122
-----------------------------


진행 중:  99%|█████████▊| 73/74 [07:38<00:06,  6.60s/it]

식당 이름: 오발탄 대치점
식당 카테고리: 곱창,막창,양
주소: 서울 강남구 도곡로93길 7
대표 메뉴: 특양구이
가격: 43,000원
방문자리뷰 :  223
블로그리뷰 :  63
-----------------------------


진행 중: 100%|██████████| 74/74 [07:44<00:00,  6.28s/it]

식당 이름: 룸카페헬로
식당 카테고리: 룸카페
주소: 서울 강남구 강남대로102길 16 까치빌딩 3층
대표 메뉴: 메뉴없음
가격: 가격없음
방문자리뷰 :  107
블로그리뷰 :  30
-----------------------------


만약에 에러나면 에러난 식당의 리뷰가 그 전 식당의 리뷰수를 따라가더라 

In [ ]:
# ## 잘 긁어왔나 확인합니다

# res_info_df.head()

In [ ]:
# ## 엑셀파일에 잘 저장합니다

# res_info_df.to_excel('매장정보.xlsx')


# --------------------------------------------------------------------------------------

# 여기서부턴 파일 옮겨서 돌려야 할 듯 합니다

# 업태가 뭐있는지 몰라서입니다  + 리뷰도 다른 파일에서 취합하는ㄱ네 맞는듯 싶습니다 


In [ ]:
# res_info_df['업태구분'].unique()

In [ ]:
# # 필요없는 카테고리(카페, 술, 출장요리 등등) 제거
# 제외 = ['바나프레소','이자카야','바(BAR)','카페,디저트','요리주점','장소대여','카페','PC','예식장']
# for i in 제외 :
#     res_info_df = res_info_df.drop(res_info_df[res_info_df['업태구분'] == i].index)

# len(res_info_df)

In [ ]:
# res_info_df

In [ ]:
# ## 한식 하나 더 만들어서 정렬되는지 확인하려고 만들었음 

# res_info_df.loc[df_test_result['업태구분']=='한식뷔페','업태구분' ] = '한식'

In [ ]:
# res_df_groupby = res_info_df.groupby('업태구분').apply(lambda x : x.sort_values(by='리뷰',ascending=False))


In [ ]:
# res_df_groupby

In [ ]:

# # 예제 데이터프레임 생성
# test = pd.DataFrame({
#     '업태구분': ['식당', '식당', '카페', '카페', '호텔', '호텔'],
#     '리뷰': [5, 4, 3, 2, 1, 6]
# })

# # 상위 n개를 뽑는 함수 정의
# def top_n(group, n=2):
#     return group.head(n)

# # '업태구분' 칼럼을 기준으로 그룹화하고, 각 그룹별로 정렬한 후 상위 2개의 데이터를 뽑아 새로운 데이터프레임으로 생성
# test_groupby = test.groupby('업태구분', group_keys=False).apply(lambda x: x.sort_values(by='리뷰', ascending=False)).groupby('업태구분').apply(top_n, n=2)

# print(test_groupby)

In [ ]:
# res_df_groupby

In [ ]:
# res_df_groupby.to_csv('매장_업태별_결과.csv')